In [1]:
# Import necessary libraries

import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [2]:
### Load Data

### Training

# Training data
x_train = np.array([np.array([1,2,3]),np.array([9,5,7]),np.array([9,4,2]),np.array([11,5,7]), np.array([11,1,15]), np.array([0,4,20]),np.array([2,3,9]),np.array([12,18,5]), np.array([0,6,1]), np.array([1,1,5])])
print("Training data is:", x_train)
print("Shape of training data:", x_train.shape)
print("*"*20)
# Training labels
labels_train = np.asarray([0,1,1,0,1,0,0,1,0,1])
print("Training data label is: ", labels_train)
print("Shape of training data labels:", labels_train.shape)
print("*"*20)
print("*"*20)
### Validation

# Validation data
x_val = np.array([np.array([4,0,0]),np.array([0,3,9]),np.array([7,7,2]),np.array([1,3,3])])
print("Shape of Validation data:", x_val.shape)
# Validation labels
labels_val = np.asarray([0,1,1,0])
print("Shape of validation data labels", labels_val.shape)

Training data is: [[ 1  2  3]
 [ 9  5  7]
 [ 9  4  2]
 [11  5  7]
 [11  1 15]
 [ 0  4 20]
 [ 2  3  9]
 [12 18  5]
 [ 0  6  1]
 [ 1  1  5]]
Shape of training data: (10, 3)
********************
Training data label is:  [0 1 1 0 1 0 0 1 0 1]
Shape of training data labels: (10,)
********************
********************
Shape of Validation data: (4, 3)
Shape of validation data labels (4,)


In [3]:
# Check if cuda is available

cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = False with num_workers = 0


In [4]:
# Dataset definition

class MLPDataset(Dataset):
    
    # load the dataset
    def __init__(self, x, y):
        
        # store the inputs and outputs
        self.X = x
        self.y = y
        
    # number of rows in the dataset
    def __len__(self):
        return len(self.y)
 
    # get a row at an index
    def __getitem__(self, index):
        
        x = torch.Tensor(self.X[index]).float() 
        y = self.y[index]
    
        return x,y

In [5]:
## Dataloaders

# Training dataloader
train_data = MLPDataset(x_train, labels_train)
train_args = dict(shuffle = True, batch_size = 2, drop_last=True)
train_loader = DataLoader(train_data, **train_args)

# Validation dataloader
val_data = MLPDataset(x_val, labels_val)
val_args = dict(shuffle = True, batch_size = 1, drop_last=True)
val_loader = DataLoader(val_data, **val_args)


In [6]:
## Model Architecture definition

class MLP(nn.Module):

    # define model elements
    def __init__(self, size):
        super(MLP, self).__init__()
        
        # Sequential model definition: Input -> Linear -> ReLU -> Linear -> Output
        
        self.model = nn.Sequential(nn.Linear(size[0], size[1]), nn.ReLU(),
                                   nn.Linear(size[1], size[2]))

    def forward(self, x):
        f
        # Model forward pass
        return self.model(x)

In [7]:
# Model
model = MLP([3,128,2])

# Define Criterion/ Loss function
criterion = nn.CrossEntropyLoss()

# Define Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

print(model)

MLP(
  (model): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)


In [8]:
# Train the model

def train_model(train_loader, model):
    training_loss = 0
    
    # Set model in 'Training mode'
    model.train()
    
    # enumerate mini batches
    for i, (inputs, targets) in enumerate(train_loader):
        
        # clear the gradients
        optimizer.zero_grad()
        
        # compute the model output
        out = model(inputs)
        
        # calculate loss
        loss = criterion(out, targets)
        
        # Backward pass
        loss.backward()
        
        # Update model weights
        optimizer.step()

        training_loss += loss.item()
    training_loss /= len(train_loader)
    return training_loss

In [9]:
# Evaluate the model

def evaluate_model(val_loader, model):
    
    predictions = []
    actuals = []
    
    # Set model in validation mode
    model.eval()
    
    for i, (inputs, targets) in enumerate(val_loader):
        
        # evaluate the model on the validation set
        out = model(inputs)
                
        # Calculate validation loss
        loss = criterion(out, targets)
        
        # retrieve numpy array
        out = out.detach().numpy()
        actual = targets.numpy()

        
        # convert to class labels
        out = np.argmax(out, axis=1)
        
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        out = out.reshape((len(out), 1))
        # store
        predictions.append(out)
        actuals.append(actual)
    
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # Calculate validation accuracy
    acc = accuracy_score(actuals, predictions)
    return acc, loss.item()
 

In [10]:
# Define number of epochs
epochs = 10

for epoch in range(epochs):
    
    # Train
    training_loss = train_model(train_loader, model)

    # Validation
    val_acc, val_loss = evaluate_model(train_loader, model)
    
    # Print log of accuracy and loss
    print("Epoch: "+str(epoch)+", Training loss: "+str(training_loss)+", Validation loss:"+str(val_loss)+
          ", Validation accuracy:"+str(val_acc*100)+"%")

Epoch: 0, Training loss: 0.8822132185101509, Validation loss:0.5599926710128784, Validation accuracy:60.0%
Epoch: 1, Training loss: 0.6252903640270233, Validation loss:0.6960655450820923, Validation accuracy:80.0%
Epoch: 2, Training loss: 0.6506033182144165, Validation loss:0.6304447650909424, Validation accuracy:50.0%
Epoch: 3, Training loss: 0.6013909876346588, Validation loss:0.3395001292228699, Validation accuracy:80.0%
Epoch: 4, Training loss: 0.5528831720352173, Validation loss:0.6480250358581543, Validation accuracy:80.0%
Epoch: 5, Training loss: 0.519448147714138, Validation loss:1.041185975074768, Validation accuracy:80.0%
Epoch: 6, Training loss: 0.5149343252182007, Validation loss:0.7843772768974304, Validation accuracy:80.0%
Epoch: 7, Training loss: 0.5013197004795075, Validation loss:0.2617728114128113, Validation accuracy:80.0%
Epoch: 8, Training loss: 0.521775758266449, Validation loss:0.7564153075218201, Validation accuracy:80.0%
Epoch: 9, Training loss: 0.4881810098886